<a href="https://colab.research.google.com/github/ShankarPoudel441/self-Sub-functions/blob/main/DataGenerator_for_UNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import os
import pandas as pd
import numpy as np

import tensorflow as tf
from sklearn.model_selection import train_test_split

import tensorflow as tf
import cv2




In [53]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
W=256
H=256

test_df_path='/content/drive/MyDrive/CBIS-DDSM/Unet_mask/mass/Image_mask_test.csv'
train_df_path="/content/drive/MyDrive/CBIS-DDSM/Unet_mask/mass/Image_mask_train.csv"

image_mask_test=pd.read_csv(test_df_path)
img_mask=cv2.imread(image_mask_test["mask_destination"][0])
img_mask.shape

(4006, 1846, 3)

In [7]:
def process_data(dataframe_path):
  df = pd.read_csv(dataframe_path)
  print(dataframe_path)
  images=df.image_file.values
  masks=df.mask_destination.values
  # print(images)
  # print(masks)

  return images, masks

In [8]:
def load_data(test_df_path,train_df_path):
  test_df_path='/content/drive/MyDrive/CBIS-DDSM/Unet_mask/mass/Image_mask_test.csv'
  train_df_path="/content/drive/MyDrive/CBIS-DDSM/Unet_mask/mass/Image_mask_train.csv"

  train_x, train_y=process_data(train_df_path)
  test_x, test_y = process_data(test_df_path)

  train_x,val_x=train_test_split(train_x, test_size=0.2, random_state=42)
  train_y,val_y=train_test_split(train_y, test_size=0.2, random_state=42)

  return (train_x, train_y), (val_x,val_y), (test_x,test_y)



In [9]:
def read_image(x):
  x=cv2.imread(x,cv2.IMREAD_COLOR)
  x = cv2.resize(x, (W,H))
  x = x/255.0
  x = x.astype(np.float32)
  return x


def read_mask(x):
  x = cv2.imread(x,cv2.IMREAD_GRAYSCALE)
  # print(x.shape)
  x = cv2.resize(x, (W,H))
  # print(x.shape)
  x = x.astype(np.int32)
  return x

In [10]:


def preprocess(x,y):
  def f(x,y):
    x= x.decode()
    y= y.decode()

    image = read_image(x)
    mask = read_mask(y)

    return image,mask

  image, mask = tf.numpy_function(f, (x,y), [tf.float32, tf.int32])

  mask = tf.one_hot(mask, 2, dtype=tf.int32)
  mask.set_shape([H,W,2])

  image.set_shape([H,W,3])
  # mask.set_shape([H,W,1])

  return image,mask
  

In [11]:
def tf_dataset(x, y, batch=8):
  dataset=tf.data.Dataset.from_tensor_slices((x,y))
  dataset = dataset.shuffle(buffer_size=5000)
  dataset = dataset.map(preprocess)
  dataset = dataset.batch(batch)
  dataset = dataset.repeat()
  dataset = dataset.prefetch(3)
  return dataset 

In [63]:
(train_x,train_y), (val_x,val_y), (test_x,test_y) = load_data(test_df_path,train_df_path)
print(f"Dataset: Train: {len(train_x)} -Valid: {len(val_x)} -Test: {len(test_x)}")
print(f"Masks: Train: {len(train_y)} -Valid: {len(val_y)} -Test:{len(test_y)}")

# read_mask(train_y[0])
dataset= tf_dataset(train_x,train_y,batch=8)
for x,y in dataset:
  print(x.shape, y.shape)  #8,256,256,3   8,256,256,2//8,256,256,1
  break


/content/drive/MyDrive/CBIS-DDSM/Unet_mask/mass/Image_mask_train.csv
/content/drive/MyDrive/CBIS-DDSM/Unet_mask/mass/Image_mask_test.csv
Dataset: Train: 916 -Valid: 230 -Test: 327
Masks: Train: 916 -Valid: 230 -Test:327
(8, 256, 256, 3) (8, 256, 256, 2)


In [64]:
from tensorflow.keras.layers import Conv2D, Input, BatchNormalization, Activation, MaxPool2D, UpSampling2D, Concatenate
from tensorflow.keras.models import Model



In [65]:
def conv_block(inputs, filters, pool = True):
  x=Conv2D(filters, 3, padding="same") (inputs)
  x=BatchNormalization()(x)
  x=Activation("relu")(x)

  x=Conv2D(filters,3,padding="same")(x)
  x=BatchNormalization()(x)
  x=Activation("relu")(x)

  if pool == True:
    p = MaxPool2D((2,2))(x)
    return x,p
  else:
    return x



In [66]:
def build_unet(shape, num_class):
  inputs=Input(shape)

  #Encoder
  x1,p1=conv_block(inputs, 16, pool=True)
  x2,p2=conv_block(p1, 32,pool=True)
  x3,p3=conv_block(p2, 48, pool=True)
  x4,p4=conv_block(p3, 64, pool=True)

  #Bridge
  b1= conv_block(p4, 128, pool=False)

  #Decoder
  u1=UpSampling2D((2,2), interpolation="bilinear")(b1)
  # print(u1.shape,x4.shape)
  c1=Concatenate()([u1, x4])
  x5=conv_block(c1, 64, pool=False)

  u2=UpSampling2D((2,2), interpolation="bilinear")(x5)
  # print(u2.shape,x3.shape)
  c2=Concatenate()([u2, x3])
  x6=conv_block(c2, 48, pool=False)

  u3=UpSampling2D((2,2), interpolation="bilinear")(x6)
  # print(u3.shape,x2.shape)
  c3=Concatenate()([u3, x2])
  x7=conv_block(c3, 32, pool=False)

  u4=UpSampling2D((2,2), interpolation="bilinear")(x7)
  # print(u4.shape,x1.shape)
  c4=Concatenate()([u4, x1])
  x8=conv_block(c4, 16, pool=False)

  #Output
  output=Conv2D(num_class, 3, padding="same", activation="softmax")(x8)

  return Model(inputs, output)

In [67]:
# model= build_unet((256,256,3),1)
# model.summary()

In [68]:
#Seeding 
np.random.seed(42)
tf.random.set_seed(42)


In [ ]:
#Dataset
(train_x,train_y), (val_x,val_y), (test_x,test_y) = load_data(test_df_path,train_df_path)
print(f"Dataset: Train: {len(train_x)} -Valid: {len(val_x)} -Test: {len(test_x)}")
print(f"Masks: Train: {len(train_y)} -Valid: {len(val_y)} -Test:{len(test_y)}")


In [71]:
#Hyperparameter

shape=(256,256,3)
num_class=2
lr=1e-2
batch_size= 8
epochs=10


# shape = (256, 256, 3)
# num_classes = 2
# lr = 1e-4
# batch_size = 8
# epochs = 10

In [72]:
#Model
model = build_unet(shape,num_class)
model.compile(loss=tf.keras.losses.CategoricalCrossentropy,
              optimizer=tf.keras.optimizers.Adam(lr))
train_dataset=tf_dataset(train_x,train_y,batch=batch_size)
val_dataset=tf_dataset(val_x,val_y, batch=batch_size)

train_steps=len(train_x)//batch_size
val_steps=len(val_x)//batch_size



model = build_unet(shape, num_classes)
model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(lr))

train_dataset=tf_dataset(train_x,train_y,batch=batch_size)
valid_dataset=tf_dataset(val_x,val_y, batch=batch_size)

train_steps = len(train_x)//batch_size
valid_steps = len(valid_x)//batch_size

In [73]:

callbacks = [
    ModelCheckpoint('model.h5', verbose=1, save_best_model=True),
    ReduceLROnPlateau(monitor="val_loss", patience=3, factor=0.1, verbose=1, min_lr=1e-7),
    EarlyStopping(monitor="val_loss", patience=5, verbose=1)
]



In [78]:
# model.fit(train_dataset,
#           steps_per_epoch = train_steps,
#           # validation_split=0.2,
#           validation_data = val_dataset,
#           # validataion_steps = val_steps,
#           epochs =epochs,
#           callbacks= callbacks
#           )

Epoch 1/10


TypeError: ignored

In [84]:
 """ Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Dataset """
path = "oxford-iiit-pet/"
(train_x,train_y), (valid_x,valid_y), (test_x,test_y) = load_data(test_df_path,train_df_path)
print(f"Dataset: Train: {len(train_x)} - Valid: {len(valid_x)} - Test: {len(test_x)}")

""" Hyperparameters """
shape = (256, 256, 3)
num_classes = 2
lr = 1e-4
batch_size = 8
epochs = 10

""" Model """
model = build_unet(shape, num_classes)
model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(lr))

train_dataset=tf_dataset(train_x,train_y,batch=batch_size)
valid_dataset=tf_dataset(val_x,val_y, batch=batch_size)

train_steps = len(train_x)//batch_size
valid_steps = len(valid_x)//batch_size

callbacks = [
    ModelCheckpoint("model.h5", verbose=1, save_best_model=True),
    ReduceLROnPlateau(monitor="val_loss", patience=3, factor=0.1, verbose=1, min_lr=1e-6),
    EarlyStopping(monitor="val_loss", patience=5, verbose=1)
]

model.fit(train_dataset,
    steps_per_epoch=train_steps,
    validation_data=valid_dataset,
    validation_steps=valid_steps,
    epochs=epochs,
    callbacks=callbacks
)

/content/drive/MyDrive/CBIS-DDSM/Unet_mask/mass/Image_mask_train.csv
/content/drive/MyDrive/CBIS-DDSM/Unet_mask/mass/Image_mask_test.csv
Dataset: Train: 916 - Valid: 230 - Test: 327
Epoch 1/10
 31/114 [=======>......................] - ETA: 11:54 - loss: 0.6885

KeyboardInterrupt: ignored